# Fake vs. Real News Classification using NLP

The goal of this project is to classify news as fake or real based on the title of the article and the article text

Dataset was pulled from kaggle: https://www.kaggle.com/datasets/jillanisofttech/fake-or-real-news

## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

## Reading in the Data

In [2]:
# Pulling the data from my github repository
!wget https://github.com/dallastealer/FakeNewsClassifier/blob/main/fake_or_real_news.csv.zip?raw=true

--2022-04-26 00:16:06--  https://github.com/dallastealer/FakeNewsClassifier/blob/main/fake_or_real_news.csv.zip?raw=true
Resolving github.com (github.com)... 52.192.72.89
Connecting to github.com (github.com)|52.192.72.89|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/dallastealer/FakeNewsClassifier/raw/main/fake_or_real_news.csv.zip [following]
--2022-04-26 00:16:07--  https://github.com/dallastealer/FakeNewsClassifier/raw/main/fake_or_real_news.csv.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dallastealer/FakeNewsClassifier/main/fake_or_real_news.csv.zip [following]
--2022-04-26 00:16:07--  https://raw.githubusercontent.com/dallastealer/FakeNewsClassifier/main/fake_or_real_news.csv.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubuserco

In [3]:
from zipfile import ZipFile
# use zipfile to extract contents from downloaded zip
file = ZipFile("fake_or_real_news.csv.zip?raw=true")
file.extractall()
file.close()

## View Data

In [4]:
df = pd.read_csv("fake_or_real_news.csv")
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [5]:
# View a sample text
df["text"].iloc[0]

'Daniel Greenfield, a Shillman Journalism Fellow at the Freedom Center, is a New York writer focusing on radical Islam. \nIn the final stretch of the election, Hillary Rodham Clinton has gone to war with the FBI. \nThe word “unprecedented” has been thrown around so often this election that it ought to be retired. But it’s still unprecedented for the nominee of a major political party to go war with the FBI. \nBut that’s exactly what Hillary and her people have done. Coma patients just waking up now and watching an hour of CNN from their hospital beds would assume that FBI Director James Comey is Hillary’s opponent in this election. \nThe FBI is under attack by everyone from Obama to CNN. Hillary’s people have circulated a letter attacking Comey. There are currently more media hit pieces lambasting him than targeting Trump. It wouldn’t be too surprising if the Clintons or their allies were to start running attack ads against the FBI. \nThe FBI’s leadership is being warned that the entir

In [6]:
# Check average length of title in words

title_lengths = [len(title.split()) for title in df["title"]]
np.mean(title_lengths)

10.496448303078138

In [7]:
# Check average length of text in words
text_lengths = [len(text.split()) for text in df["text"]]
np.mean(text_lengths)

776.3007103393844

## Prepare Data For Model

In [8]:
# Append Title to the front of each article

X = []
y = []

for index, row in df.iterrows():
  X.append((row["title"] + ": " + row["text"]).lower())
  y.append(row["label"])

X[0], y[:5]

('you can smell hillary’s fear: daniel greenfield, a shillman journalism fellow at the freedom center, is a new york writer focusing on radical islam. \nin the final stretch of the election, hillary rodham clinton has gone to war with the fbi. \nthe word “unprecedented” has been thrown around so often this election that it ought to be retired. but it’s still unprecedented for the nominee of a major political party to go war with the fbi. \nbut that’s exactly what hillary and her people have done. coma patients just waking up now and watching an hour of cnn from their hospital beds would assume that fbi director james comey is hillary’s opponent in this election. \nthe fbi is under attack by everyone from obama to cnn. hillary’s people have circulated a letter attacking comey. there are currently more media hit pieces lambasting him than targeting trump. it wouldn’t be too surprising if the clintons or their allies were to start running attack ads against the fbi. \nthe fbi’s leadership

In [9]:
# Check what text length covers 95% of the data

lengths = [len(article.split()) for article in X]
coverage_length = np.percentile(lengths, 95)
print(f"95% of the articles are {int(coverage_length)} words in length or less")

95% of the articles are 2036 words in length or less


## Splitting Data into Training and Testing Sets

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
len(X_train), len(y_train), len(X_test), len(y_test)

(5384, 5384, 951, 951)

In [11]:
# Turn labels into 1 and 0
y_labels = {"FAKE": 0, "REAL": 1}
y_train = [y_labels[label] for label in y_train]
y_test = [y_labels[label] for label in y_test]
y_train[:5], y_test[:5]

([1, 0, 0, 0, 0], [0, 0, 0, 0, 1])

## Text Vectorizer

In [12]:
from tensorflow.keras.layers import TextVectorization

In [13]:
vectorizer = TextVectorization(max_tokens=60000, #Vocabulary size, may need to increase
                               output_mode="int",
                               output_sequence_length=int(coverage_length))

In [14]:
# Fit vectorizer to training data
vectorizer.adapt(X_train)

In [15]:
# Test that vectorizer is working

print(f"10 most common words: {vectorizer.get_vocabulary()[:10]}")
print(f"10 least common words: {vectorizer.get_vocabulary()[-10:]}")
print(f"Vocabulary size: {len(vectorizer.get_vocabulary())}")

10 most common words: ['', '[UNK]', 'the', 'to', 'of', 'and', 'a', 'in', 'that', 'is']
10 least common words: ['\x93very', '\x93trump', '\x93tradecraft', '\x93trace', '\x93toughness\x94', '\x93too', '\x93this', '\x93they\x92re', '\x93these', '\x93there\x92s']
Vocabulary size: 60000


## Custom Embedding Layer

In [16]:
from tensorflow.keras.layers import Embedding

embedding = Embedding(input_dim=60000, #Should match vocab size
                      output_dim=128,
                      input_length=int(coverage_length))

In [17]:
# Test Embedding
embedding(vectorizer(X_train[0]))

<tf.Tensor: shape=(2036, 128), dtype=float32, numpy=
array([[ 0.04849455,  0.03798131, -0.01827631, ...,  0.04722777,
        -0.01475282, -0.00950872],
       [ 0.03924265, -0.01814746,  0.02333278, ..., -0.02782877,
         0.02108515,  0.03726808],
       [ 0.01381816,  0.04656034, -0.04170482, ..., -0.03166689,
         0.00227934, -0.01214473],
       ...,
       [-0.04459558, -0.02250531, -0.03227811, ..., -0.01424537,
        -0.01146477,  0.03793548],
       [-0.04459558, -0.02250531, -0.03227811, ..., -0.01424537,
        -0.01146477,  0.03793548],
       [-0.04459558, -0.02250531, -0.03227811, ..., -0.01424537,
        -0.01146477,  0.03793548]], dtype=float32)>

## Create Baseline with Sklearn TF-IDF and Naive Bayes

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
  ("tfidf", TfidfVectorizer()), # Words to numbers
  ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [19]:
# Evaluate Baseline Accuracy
baseline_accuracy = model_0.score(X_test, y_test)
print(f"Baseline model performed with accuracy: {baseline_accuracy.round(2)}")

Baseline model performed with accuracy: 0.83


## Turn data into tf dataset

In [20]:
from tensorflow.python.data.ops.dataset_ops import AUTOTUNE
training_data = tf.data.Dataset.from_tensor_slices((X_train, y_train))
testing_data = tf.data.Dataset.from_tensor_slices((X_test, y_test))

# Turn data into batches and use prefetching to speed up training process
training_data = training_data.batch(32).prefetch(AUTOTUNE)
testing_data = testing_data.batch(32).prefetch(AUTOTUNE)

## First Model: Simple Dense Model

In [21]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string") # Input layer

x = vectorizer(inputs) # Turn words into numbers
x = embedding(x) # Turn each vectorized word into a vector of length 128
x = layers.Dense(64, activation="relu")(x) # Dense Layer
x = layers.GlobalAveragePooling1D()(x) # Average pooling to reduce dimensionality
outputs = layers.Dense(1, activation="sigmoid")(x) # Output layer

model_1 = tf.keras.Model(inputs, outputs)
model_1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 2036)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 2036, 128)         7680000   
                                                                 
 dense (Dense)               (None, 2036, 64)          8256      
                                                                 
 global_average_pooling1d (G  (None, 64)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense_1 (Dense)             (None, 1)                 65    

In [22]:
# Compile first model
model_1.compile(loss="binary_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])

In [23]:
# Fit first model
history_1 = model_1.fit(training_data,
                        epochs=5,
                        steps_per_epoch=len(training_data),
                        validation_data=testing_data,
                        validation_steps=int(0.5 * len(testing_data)))

Epoch 1/5
169/169 [==============================] - 5s 19ms/step - loss: 0.6394 - accuracy: 0.6724 - val_loss: 0.5512 - val_accuracy: 0.7437
Epoch 2/5
169/169 [==============================] - 3s 18ms/step - loss: 0.4129 - accuracy: 0.8418 - val_loss: 0.3583 - val_accuracy: 0.8375
Epoch 3/5
169/169 [==============================] - 3s 18ms/step - loss: 0.2650 - accuracy: 0.8965 - val_loss: 0.2821 - val_accuracy: 0.8792
Epoch 4/5
169/169 [==============================] - 3s 19ms/step - loss: 0.1889 - accuracy: 0.9326 - val_loss: 0.2427 - val_accuracy: 0.8979
Epoch 5/5
169/169 [==============================] - 3s 19ms/step - loss: 0.1399 - accuracy: 0.9519 - val_loss: 0.2200 - val_accuracy: 0.9062


In [24]:
model_1_accuracy = model_1.evaluate(testing_data)

30/30 [==============================] - 0s 16ms/step - loss: 0.2006 - accuracy: 0.9096


In [25]:
print(f"Baseline accuracy: {(baseline_accuracy * 100).round(2)}%")
print(f"Model 1 accuracy: {np.round(model_1_accuracy[1] * 100, 2)}%")

Baseline accuracy: 82.75%
Model 1 accuracy: 90.96%


## Sample Predictions from Model 1

In [26]:
labels_from_preds = {0: "FAKE", 1: "REAL"}

In [27]:
article_to_test = np.random.randint(0, len(X_test))
print(f"Article:\n {X_test[article_to_test][:200]}...")
pred_prob = model_1.predict([X_test[article_to_test]])
pred_prob_rounded = int(np.round(pred_prob).squeeze())
print(f"Model prediction: {labels_from_preds[pred_prob_rounded]}")
print(f"Actual Label: {labels_from_preds[y_test[article_to_test]]}")


Article:
 hillary friend bribed fbi agent and his wife: email 

wsj the political organization of virginia gov. terry mcauliffe, an influential democrat with longstanding ties to bill and hillary clinton, gave ...
Model prediction: FAKE
Actual Label: FAKE


## Model 2: LSTM

In [28]:
inputs = layers.Input(shape=(1,), dtype="string")
x = vectorizer(inputs)
x = embedding(x)

x = layers.LSTM(128)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_2 = tf.keras.Model(inputs, outputs)
model_2.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 2036)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 2036, 128)         7680000   
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 7,811,713
Trainable params: 7,811,713
Non-trainable params: 0
_________________________________________________

In [29]:
model_2.compile(loss="binary_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])

In [30]:
history_2 = model_2.fit(training_data,
                        epochs=5,
                        validation_data=testing_data,
                        validation_steps=len(testing_data))

Epoch 1/5
169/169 [==============================] - 21s 107ms/step - loss: 0.6845 - accuracy: 0.5254 - val_loss: 0.4758 - val_accuracy: 0.8349
Epoch 2/5
169/169 [==============================] - 19s 113ms/step - loss: 0.6155 - accuracy: 0.6566 - val_loss: 0.6859 - val_accuracy: 0.5268
Epoch 3/5
169/169 [==============================] - 18s 107ms/step - loss: 0.6835 - accuracy: 0.5089 - val_loss: 0.6896 - val_accuracy: 0.5047
Epoch 4/5
169/169 [==============================] - 18s 107ms/step - loss: 0.6730 - accuracy: 0.5736 - val_loss: 0.5956 - val_accuracy: 0.7129
Epoch 5/5
169/169 [==============================] - 18s 109ms/step - loss: 0.5824 - accuracy: 0.6731 - val_loss: 0.5838 - val_accuracy: 0.6719


In [31]:
model_2_accuracy = model_2.evaluate(testing_data)
print(f"Baseline accuracy: {(baseline_accuracy * 100).round(2)}%")
print(f"Model 1 accuracy: {np.round(model_1_accuracy[1] * 100, 2)}%")
print(f"Model 1 accuracy: {np.round(model_2_accuracy[1] * 100, 2)}%")

30/30 [==============================] - 1s 49ms/step - loss: 0.5838 - accuracy: 0.6719
Baseline accuracy: 82.75%
Model 1 accuracy: 90.96%
Model 1 accuracy: 67.19%


## Model 3: Universal Sentence Encoder from tensorflow hub

In [32]:
import tensorflow_hub as hub

In [33]:
universal_sentence_encoder = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4", #Link from tensorflow hub
                                            trainable=False,
                                            input_shape=[],
                                            dtype=tf.string)

In [34]:
model_3 = tf.keras.Sequential([
  universal_sentence_encoder, #Sentence encoder gets rid of need for vectorizer and embeddings
  layers.Dense(64, activation="relu"),
  layers.Dense(1, activation="sigmoid")
])

model_3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 512)               256797824 
                                                                 
 dense_3 (Dense)             (None, 64)                32832     
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [35]:
model_3.compile(loss="binary_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])

In [36]:
history_3 = model_3.fit(training_data,
                        epochs=5,
                        steps_per_epoch=len(training_data),
                        validation_data=testing_data,
                        validation_steps=int(0.15* len(testing_data)))

Epoch 1/5
169/169 [==============================] - 25s 133ms/step - loss: 0.4702 - accuracy: 0.8022 - val_loss: 0.3185 - val_accuracy: 0.8672
Epoch 2/5
169/169 [==============================] - 22s 129ms/step - loss: 0.3219 - accuracy: 0.8637 - val_loss: 0.2743 - val_accuracy: 0.8828
Epoch 3/5
169/169 [==============================] - 22s 129ms/step - loss: 0.2914 - accuracy: 0.8776 - val_loss: 0.2583 - val_accuracy: 0.8750
Epoch 4/5
169/169 [==============================] - 22s 129ms/step - loss: 0.2753 - accuracy: 0.8863 - val_loss: 0.2490 - val_accuracy: 0.8750
Epoch 5/5
169/169 [==============================] - 22s 130ms/step - loss: 0.2642 - accuracy: 0.8936 - val_loss: 0.2433 - val_accuracy: 0.8750


In [37]:
model_3_accuracy = model_3.evaluate(testing_data)
print(f"Baseline accuracy: {(baseline_accuracy * 100).round(2)}%")
print(f"Model 1 accuracy: {np.round(model_1_accuracy[1] * 100, 2)}%")
print(f"Model 3 accuracy: {np.round(model_3_accuracy[1] * 100, 2)}%")

30/30 [==============================] - 5s 158ms/step - loss: 0.2426 - accuracy: 0.8959
Baseline accuracy: 82.75%
Model 1 accuracy: 90.96%
Model 3 accuracy: 89.59%


## Model 4: Same as model 3 but double the epochs

In [38]:
model_4 = tf.keras.Sequential([
  universal_sentence_encoder,
  layers.Dense(64, activation="relu"),
  layers.Dense(1, activation="sigmoid")
])

model_4.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 512)               256797824 
                                                                 
 dense_5 (Dense)             (None, 64)                32832     
                                                                 
 dense_6 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [39]:
model_4.compile(loss="binary_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])

In [40]:
history_4 = model_4.fit(training_data,
                        epochs=10,
                        steps_per_epoch=len(training_data),
                        validation_data=testing_data,
                        validation_steps=int(0.15*len(testing_data)))

Epoch 1/10
169/169 [==============================] - 23s 132ms/step - loss: 0.4736 - accuracy: 0.7957 - val_loss: 0.3200 - val_accuracy: 0.8750
Epoch 2/10
169/169 [==============================] - 21s 125ms/step - loss: 0.3218 - accuracy: 0.8653 - val_loss: 0.2761 - val_accuracy: 0.8906
Epoch 3/10
169/169 [==============================] - 21s 126ms/step - loss: 0.2918 - accuracy: 0.8780 - val_loss: 0.2610 - val_accuracy: 0.8750
Epoch 4/10
169/169 [==============================] - 22s 128ms/step - loss: 0.2762 - accuracy: 0.8858 - val_loss: 0.2526 - val_accuracy: 0.8750
Epoch 5/10
169/169 [==============================] - 21s 127ms/step - loss: 0.2655 - accuracy: 0.8923 - val_loss: 0.2473 - val_accuracy: 0.8750
Epoch 6/10
169/169 [==============================] - 22s 129ms/step - loss: 0.2572 - accuracy: 0.8958 - val_loss: 0.2439 - val_accuracy: 0.8828
Epoch 7/10
169/169 [==============================] - 22s 128ms/step - loss: 0.2503 - accuracy: 0.8980 - val_loss: 0.2418 - val_ac

In [41]:
model_4_accuracy = model_4.evaluate(testing_data)

30/30 [==============================] - 5s 154ms/step - loss: 0.2342 - accuracy: 0.8980


In [42]:
print(f"Baseline accuracy: {(baseline_accuracy * 100).round(2)}%")
print(f"Model 1 accuracy: {np.round(model_1_accuracy[1] * 100, 2)}%")
print(f"Model 3 accuracy: {np.round(model_3_accuracy[1] * 100, 2)}%")
print(f"Model 4 accuracy: {np.round(model_4_accuracy[1] * 100, 2)}%")

Baseline accuracy: 82.75%
Model 1 accuracy: 90.96%
Model 3 accuracy: 89.59%
Model 4 accuracy: 89.8%


## Model 5: First model but twice as many epochs

In [43]:
inputs = layers.Input(shape=(1,), dtype="string")

x = vectorizer(inputs)
x = embedding(x)
x = layers.Dense(64, activation="relu")(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_5 = tf.keras.Model(inputs, outputs)
model_5.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 2036)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 2036, 128)         7680000   
                                                                 
 dense_7 (Dense)             (None, 2036, 64)          8256      
                                                                 
 global_average_pooling1d_1   (None, 64)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_8 (Dense)             (None, 1)                 65  

In [44]:
model_5.compile(loss="binary_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])

In [45]:
history_5 = model_5.fit(training_data,
                        epochs=10,
                        steps_per_epoch=len(training_data),
                        validation_data=testing_data,
                        validation_steps=int(0.15*len(testing_data)))

Epoch 1/10
169/169 [==============================] - 3s 15ms/step - loss: 0.4987 - accuracy: 0.8518 - val_loss: 0.3474 - val_accuracy: 0.8672
Epoch 2/10
169/169 [==============================] - 2s 14ms/step - loss: 0.2307 - accuracy: 0.9328 - val_loss: 0.2489 - val_accuracy: 0.9062
Epoch 3/10
169/169 [==============================] - 2s 14ms/step - loss: 0.1501 - accuracy: 0.9515 - val_loss: 0.2209 - val_accuracy: 0.9062
Epoch 4/10
169/169 [==============================] - 2s 14ms/step - loss: 0.1104 - accuracy: 0.9645 - val_loss: 0.2100 - val_accuracy: 0.9297
Epoch 5/10
169/169 [==============================] - 2s 14ms/step - loss: 0.0850 - accuracy: 0.9768 - val_loss: 0.2066 - val_accuracy: 0.9375
Epoch 6/10
169/169 [==============================] - 2s 14ms/step - loss: 0.0669 - accuracy: 0.9859 - val_loss: 0.2081 - val_accuracy: 0.9297
Epoch 7/10
169/169 [==============================] - 2s 14ms/step - loss: 0.0534 - accuracy: 0.9902 - val_loss: 0.2121 - val_accuracy: 0.9297

In [46]:
model_5_accuracy = model_5.evaluate(testing_data)

30/30 [==============================] - 0s 11ms/step - loss: 0.1834 - accuracy: 0.9464


In [47]:
#model_5.save("PUT SAVE PATH HERE")

## Model 6: Testing if same (or better) results can be achieved with even smaller model

In [48]:
inputs = layers.Input(shape=(1,), dtype="string")

x = vectorizer(inputs)
x = embedding(x)
x = layers.Dense(16, activation="relu")(x)
x = layers.Dense(8, activation="relu")(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_6 = tf.keras.Model(inputs, outputs)
model_6.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 2036)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 2036, 128)         7680000   
                                                                 
 dense_9 (Dense)             (None, 2036, 16)          2064      
                                                                 
 dense_10 (Dense)            (None, 2036, 8)           136       
                                                                 
 global_average_pooling1d_2   (None, 8)                0         
 (GlobalAveragePooling1D)                                  

In [49]:
model_6.compile(loss="binary_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])

In [50]:
history_6 = model_6.fit(training_data,
                        epochs=8,
                        steps_per_epoch=len(training_data),
                        validation_data=testing_data,
                        validation_steps=len(testing_data))

Epoch 1/8
169/169 [==============================] - 4s 17ms/step - loss: 0.6149 - accuracy: 0.8289 - val_loss: 0.4755 - val_accuracy: 0.8601
Epoch 2/8
169/169 [==============================] - 3s 16ms/step - loss: 0.2823 - accuracy: 0.9469 - val_loss: 0.2368 - val_accuracy: 0.9138
Epoch 3/8
169/169 [==============================] - 3s 16ms/step - loss: 0.1296 - accuracy: 0.9773 - val_loss: 0.1859 - val_accuracy: 0.9474
Epoch 4/8
169/169 [==============================] - 3s 16ms/step - loss: 0.0813 - accuracy: 0.9913 - val_loss: 0.1709 - val_accuracy: 0.9485
Epoch 5/8
169/169 [==============================] - 3s 16ms/step - loss: 0.0581 - accuracy: 0.9955 - val_loss: 0.1677 - val_accuracy: 0.9516
Epoch 6/8
169/169 [==============================] - 3s 16ms/step - loss: 0.0440 - accuracy: 0.9967 - val_loss: 0.1688 - val_accuracy: 0.9527
Epoch 7/8
169/169 [==============================] - 3s 16ms/step - loss: 0.0346 - accuracy: 0.9965 - val_loss: 0.1729 - val_accuracy: 0.9537
Epoch 

In [51]:
model_6_accuracy = model_6.evaluate(testing_data)
print(f"Baseline accuracy: {(baseline_accuracy * 100).round(2)}%")
print(f"Model 1 accuracy: {np.round(model_1_accuracy[1] * 100, 2)}%")
print(f"Model 3 accuracy: {np.round(model_3_accuracy[1] * 100, 2)}%")
print(f"Model 4 accuracy: {np.round(model_6_accuracy[1] * 100, 2)}%")

30/30 [==============================] - 0s 12ms/step - loss: 0.1786 - accuracy: 0.9516
Baseline accuracy: 82.75%
Model 1 accuracy: 90.96%
Model 3 accuracy: 89.59%
Model 4 accuracy: 95.16%


Overall, it appears that a simple dense model outperforms LSTM based models. Additionally, using a custom embedding layer produces better results than the pretrained universal sentence encoder.

In [52]:
#model_6.save("PUT SAVE PATH HERE")

## Writing a function to see the model's predictions

In [53]:
def predict_news(text, title=None, model=model_6):
  """
  Function to predict whether the passed text is fake or real news

  Parameters:
  text: contents of the article
  title (optional): title of the article, if passed it will be prepended to the text
  model (optional): the model with which to make predictions. Default is model_6

  Returns:
  A tuple with the first index being the predicted label, and the second index being the prediction probability
  """
  text = text.lower()
  labels = {0: "FAKE", 1: "REAL"}
  if title:
    title = title.lower()
    text = title + ": " + text
  prediction_probability = model.predict([text])
  prediction_rounded = np.round(prediction_probability.squeeze())
  prediction_label = labels[prediction_rounded]
  return (prediction_label, np.round(prediction_probability.squeeze(), 4))

In [54]:
test_text = """ Try it out! Paste the text from an article you want to test here
"""
test_title = "Paste the title of the article you want to test here"

In [55]:
# If you've pasted in your article and title, uncomment the line below to see the prediction!

#predict_news(test_text, title=test_title)